# Import Library

In [2]:
## single frame analysis: whether there are large spatta?
import cv2
from IPython import display
from matplotlib import pyplot as plt
import time
from numpy import dtype,uint8
import scipy.ndimage as ndimage
import glob
import pandas as pd
import numpy as np
import os

# Load data

In [12]:
path =  r"C:\Users\Furukawa\Desktop\20230302\data.xlsx"
target_df = pd.read_excel(path,usecols="B:P")
target_df

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1053.0,2130.0,5429.0,1036.0,2089.0,5371.0,1153.0,2324.0,5826.0,1126.0,2273.0,4929.0,1150.0,2309.0,4476.0
2,1427.0,2879.0,6928.0,1410.0,2838.0,6870.0,1527.0,3073.0,7325.0,1500.0,3022.0,6428.0,1524.0,3058.0,5975.0
3,375.0,750.0,1500.0,375.0,750.0,1500.0,375.0,750.0,1500.0,375.0,750.0,1500.0,375.0,750.0,1500.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7306.0,NaN,NaN,NaN,NaN,NaN,NaN
98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7306.0,NaN,NaN,NaN,NaN,NaN,NaN
99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7314.0,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7323.0,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
target_value = target_df.values
print(target_value.shape)

(102, 15)


In [14]:
print(target_value[15,0])

-1.0


#Extract data

##Extract Frame data

In [30]:
frame_data = []
for i in range(target_value.shape[1]): # length of target value
  frame_data.append([int(target_value[1][i]),int(target_value[2,i])])
print(frame_data)

[[1053, 1427], [2130, 2879], [5429, 6928], [1036, 1410], [2089, 2838], [5371, 6870], [1153, 1527], [2324, 3073], [5826, 7325], [1126, 1500], [2273, 3022], [4929, 6428], [1150, 1524], [2309, 3058], [4476, 5975]]


##Extract spatter frames

In [46]:
def spatterframe(data):
  frame_data =[]
  for j in range(data.shape[1]):
    temp_data=[]
    i= 7
    while data[i][j] != -1:
        if int(data[i][j]) not in temp_data:
          temp_data.append(int(data[i,j]))
        i+=1
    print(temp_data)
    frame_data.append(temp_data)
  #frame_data = np.array(frame_data)
  return frame_data

In [47]:
frame_spatter = spatterframe(target_value)

[1067, 1082, 1231, 1262, 1346, 1376, 1403]
[2148, 2155, 2166, 2184, 2193, 2203, 2216, 2220, 2240, 2272, 2286, 2331, 2334, 2357, 2370, 2391, 2399, 2491, 2515, 2580, 2611, 2643, 2668, 2695, 2715, 2735, 2741, 2752, 2758, 2765, 2767, 2790, 2799, 2852, 2864]
[5440, 5476, 5480, 5530, 5591, 5632, 5688, 5690, 5702, 5763, 5787, 5801, 5824, 5826, 5827, 5828, 5890, 5916, 5924, 6000, 6132, 6159, 6188, 6235, 6248, 6256, 6268, 6286, 6343, 6370, 6385, 6411, 6431, 6470, 6481, 6501, 6554, 6634, 6647, 6649, 6660, 6674, 6717, 6741, 6775, 6788, 6802, 6808, 6875]
[1097, 1166, 1327]
[2177, 2185, 2211, 2284, 2342, 2348, 2365, 2374, 2390, 2397, 2410, 2417, 2494, 2655, 2670, 2685, 2691, 2703, 2706, 2740, 2751, 2758, 2788, 2791, 2833, 2838]
[5391, 5392, 5424, 5483, 5520, 5578, 5731, 5757, 5781, 5911, 5985, 6126, 6311, 6323, 6366, 6373, 6430, 6503, 6506, 6590, 6700, 6737, 6770, 6785, 6789, 6822, 6842, 6860]
[1157, 1169, 1175, 1228, 1269, 1278, 1307, 1319, 1330, 1387, 1393, 1398, 1440, 1503]
[2338, 2364, 2415, 24

#movie2images

##Decide cropping area

In [20]:
 def cropping_area(movie_idx,frame_data = frame_data,x_center=940,y_center=410,w=100,h=100):
  #load data
  path = r"C:\Users\Furukawa\Desktop\20230302\230302_{}.avi".format(movie_idx)
  cap = cv2.VideoCapture(path)

  index = 0
  count = 0

  start_frame = frame_data[movie_idx - 1][0]
  end_frame = frame_data[movie_idx - 1][1]
  h_size = w
  w_size = h
  ret = True
  #get image frames
  while ret :
    print("Frame: "+ str(index))
    #フレーム情報取得
    ret, img = cap.read()
    index += 1
    if index >= start_frame and index <= end_frame:
      if count %50 == 0:
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        cv2_imshow(gray)
        gray = gray[(y_center - h_size):(y_center + h_size),(x_center - w_size):(x_center + w_size)]
        print(gray.shape)
        cv2_imshow(gray)
      count += 1
      if count >250:
        break
    elif index > end_frame :
      break

In [ ]:
cropping_area(movie_idx=15,x_center=880,y_center=340,w=120,h=120)

#cropping list
- (x_center,y_center,w_size,h_size)

In [24]:
cropping_list = [[900,418,120,120],[915,418,120,120],[None,None,None,None],[None,None,None,None],[None,None,None,None],[945,410,120,120],[840,410,120,120],[885,410,120,120],[990,410,120,120],[750,345,120,120],[820,335,120,120],[910,300,120,120],[740,350,120,120],[815,350,120,120],[890,340,120,120]]
[a,b,c,d] = cropping_list[0]
print(a,b,c,d)

900 418 120 120


In [ ]:
print(frame_spatter[0])

[1067, 1082, 1231, 1262, 1346, 1376, 1403]


In [40]:
import csv
import os

def make_dataset(video_idx:int,frame_data:list,frame_spatter:list,cropping_list:list,first = False)->None:
  """make dataset for 20230302
  Args:
    video_idx(int) : video index ( 20230302_{video_idx}.avi )
    frame_data(list) : frame data ( start frame, end frame )
    frame_spatter(list) : spatter frame index
    cropping_list(list) : cropping area (center_x, center_y, w_size, h_size)
    first (bool) : if rewrite the previous data
  """
  #load data
  path = r"C:\Users\Furukawa\Desktop\20230302\230302_{}.avi".format(video_idx)
  cap = cv2.VideoCapture(path)

  #make directory
  base_dir = r"C:\Users\Furukawa\Desktop\20230302"
  img_dir = r"C:\Users\Furukawa\Documents\TechAssistant\dataset\imgs\20230302\20230302_{}".format(video_idx)
  file_dir = os.path.join(base_dir,"dataset")
  if not os.path.exists(img_dir):
    os.makedirs(dir)
  #if not os.path.exists(file_dir):
  #  os.makedirs(file_dir)

  #csv file name
  #filename = "{}_img.csv".format(video_idx)
  #file_name = os.path.join(file_dir,filename)
  #rewrite csv file
  #if first:
  #  with open(file_name,"w") as f:
  #      writer = csv.writer(f)
  #      writer.writerow(["label","img"])

  #l
  start_frame = frame_data[video_idx-1][0]
  end_frame = frame_data[video_idx-1][1]
  index = 0
  count = 0
  [x_center,y_center,w_size,h_size] = cropping_list[video_idx-1]
  ret = True
  total_frames = end_frame - start_frame

  #get image frames
  while ret :
      #フレーム情報取得
      ret, img = cap.read()
      index += 1
      #extract target frames
      if index >= start_frame and index <= end_frame:
        count += 1
        #progress bar
        if count % 50 == 0:
          print(round(count/total_frames*100,0),"% progressed")
        
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        ### crop images
        gray = gray[(y_center - h_size) : (y_center + h_size), (x_center - w_size) : (x_center + w_size)]
        idx = "{:04d}.jpg".format(index)
        file = os.path.join(dir,idx)
        cv2.imwrite(file,gray)
        """
        idx = "{:04d}.jpg".format(count)
        file = os.path.join(dir,idx)
        if index in frame_spatter[video_idx-1]:
          with open(file_name,"a",newline='') as f:
                writer = csv.writer(f)
                writer.writerow([1,file])
        else:
          with open(file_name,"a",newline='') as f:
                writer = csv.writer(f)
                writer.writerow([0,file])
      elif index > end_frame:
        print("100% finished")
        break
      """

In [41]:
for i in range(1,16):
  if i != 3 or i != 4 or i != 5 or i != 9 or i != 12 or i != 15 :
    make_dataset(video_idx = i,frame_data = frame_data,frame_spatter = frame_spatter,cropping_list = cropping_list,first=True)
  else:
    continue

13.0 % progressed
27.0 % progressed
40.0 % progressed
53.0 % progressed
67.0 % progressed
80.0 % progressed
94.0 % progressed
7.0 % progressed
13.0 % progressed
20.0 % progressed
27.0 % progressed
33.0 % progressed
40.0 % progressed
47.0 % progressed
53.0 % progressed
60.0 % progressed
67.0 % progressed
73.0 % progressed
80.0 % progressed
87.0 % progressed
93.0 % progressed
100.0 % progressed
3.0 % progressed
7.0 % progressed
10.0 % progressed
13.0 % progressed
17.0 % progressed
20.0 % progressed
23.0 % progressed
27.0 % progressed
30.0 % progressed
33.0 % progressed
37.0 % progressed
40.0 % progressed
43.0 % progressed
47.0 % progressed
50.0 % progressed
53.0 % progressed
57.0 % progressed
60.0 % progressed
63.0 % progressed
67.0 % progressed
70.0 % progressed
73.0 % progressed
77.0 % progressed
80.0 % progressed
83.0 % progressed
87.0 % progressed
90.0 % progressed
93.0 % progressed
97.0 % progressed
100.0 % progressed
13.0 % progressed
27.0 % progressed
40.0 % progressed
53.0 % prog

In [36]:
import csv
import os

def make_dataset(video_idx:int,frame_data:list,frame_spatter:list,cropping_list:list,first = False)->None:
  """make dataset for 20230302
  Args:
    video_idx(int) : video index ( 20230302_{video_idx}.avi )
    frame_data(list) : frame data ( start frame, end frame )
    frame_spatter(list) : spatter frame index
    cropping_list(list) : cropping area (center_x, center_y, w_size, h_size)
    first (bool) : if rewrite the previous data
  """
  #load data
  path = r"C:\Users\Furukawa\Desktop\20230302\230302_{}.avi".format(video_idx)
  cap = cv2.VideoCapture(path)

  #make directory
  base_dir = r"C:\Users\Furukawa\Desktop\20230302"
  dir = r"C:\Users\Furukawa\Documents\TechAssistant\dataset\imgs\20230302\20230302_{}".format(video_idx)
  file_dir = os.path.join(base_dir,"dataset")
  if not os.path.exists(img_dir):
    os.makedirs(img_dir)
  #if not os.path.exists(file_dir):
  #  os.makedirs(file_dir)

  #csv file name
  #filename = "{}_img.csv".format(video_idx)
  #file_name = os.path.join(file_dir,filename)
  #rewrite csv file
  #if first:
  #  with open(file_name,"w") as f:
  #      writer = csv.writer(f)
  #      writer.writerow(["label","img"])

  #l
  start_frame = frame_data[video_idx-1][0]
  end_frame = frame_data[video_idx-1][1]
  index = 0
  count = 0
  [x_center,y_center,w_size,h_size] = cropping_list[video_idx-1]
  ret = True
  total_frames = end_frame - start_frame

  #get image frames
  print(f":: {video_idx} proessed ::")
  print("center position : ", (x_center,y_center))
  while ret :
      #フレーム情報取得
      ret, img = cap.read()
      index += 1
      #extract target frames
      if index >= start_frame and index <= end_frame:
        count += 1
        if video_idx == 9:
          if count >= 800:
            if count % 33 == 0:
              x_center -= 7
              print("center X : ",x_center)
        if video_idx == 12:
          if count >= 500:
            if count % 33 == 0:
              x_center -= 5
              print("center X : ",x_center)
        if video_idx == 15:
          if count >= 900:
            if count % 33 == 0:
              x_center -= 5
              print("center X : ",x_center)
        #progress bar
        if count % 50 == 0:
          print(round(count/total_frames*100,0),"% progressed")
        
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        ### crop images
        gray = gray[(y_center - h_size) : (y_center + h_size), (x_center - w_size) : (x_center + w_size)]
        idx = "{:04d}.jpg".format(index)
        file = os.path.join(img_dir,idx)
        cv2.imwrite(file,gray)
        """
        idx = "{:04d}.jpg".format(count)
        file = os.path.join(img_dir,idx)
        if index in frame_spatter[video_idx-1]:
          with open(file_name,"a",newline='') as f:
                writer = csv.writer(f)
                writer.writerow([1,file])
        else:
          with open(file_name,"a",newline='') as f:
                writer = csv.writer(f)
                writer.writerow([0,file])
      elif index > end_frame:
        print("100% finished")
        break
      """

In [39]:
index_revise = [9]
for i in index_revise:
    make_dataset(video_idx = i,frame_data = frame_data,frame_spatter = frame_spatter,cropping_list = cropping_list,first=True)

:: 9 proessed ::
center position :  (990, 410)
3.0 % progressed
7.0 % progressed
10.0 % progressed
13.0 % progressed
17.0 % progressed
20.0 % progressed
23.0 % progressed
27.0 % progressed
30.0 % progressed
33.0 % progressed
37.0 % progressed
40.0 % progressed
43.0 % progressed
47.0 % progressed
50.0 % progressed
53.0 % progressed
center X :  985
57.0 % progressed
center X :  980
center X :  975
60.0 % progressed
center X :  970
63.0 % progressed
center X :  965
center X :  960
67.0 % progressed
center X :  955
70.0 % progressed
center X :  950
center X :  945
73.0 % progressed
center X :  940
77.0 % progressed
center X :  935
center X :  930
80.0 % progressed
center X :  925
83.0 % progressed
center X :  920
center X :  915
87.0 % progressed
center X :  910
90.0 % progressed
center X :  905
center X :  900
93.0 % progressed
center X :  895
97.0 % progressed
center X :  890
center X :  885
100.0 % progressed


# Make Dataset for MAE :: Spatter frame Creation

In [109]:
import random

class Dataset():
    def __init__(self,type_dataset,
                 height=240,width=240,x_patch=3,y_patch=3,
                 frame_data = frame_data,frame_spatter=frame_spatter,
                 src = r"C:\Users\Furukawa\Documents\TechAssistant\dataset\imgs\20230302"):
      self.type_dataset = type_dataset #"concat_MAE","concat_ViT","resnet"
      self.src_img_dir = src # source img directory
      self.height = height #height of original img
      self.width = width # width of original img
      self.x_patch = x_patch # num of patch in x axis
      self.y_patch = y_patch # num of patch in y axis
      self.frame_data = frame_data # start frame and end frame
      self.frame_spatter = frame_spatter # spatter frame

      self.H = height*y_patch #height of concatenated img
      self.W = width*x_patch # width of concatenated img 


    def concatImg(self,index,root_path,save_dir,INVERSE = False):
      """concatenate images
      Args:
        index: index for spatter frame, so index is the latest index in 9 frames, ie. index, index-1,index-2,..., index-8
        src_img_dir : source directory of cropped imgs
        save_dir : save dir for concatenate images
        x_patch, y_patch : number of patches in x and y axis
        INVERSE : patch order. If INVERSE is True, new to old is applied
      Return:
        file_path : path for concat img
        write concatenated images into designated directory
      """
      concat = np.zeros((self.H,self.W,1))
      for i in range(9):
          file = "{:04d}.jpg".format(index-i) #from new to old
          path = os.path.join(root_path,file)
          img = cv2.imread(path)
          img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #gray scale
          img = np.expand_dims(np.array(img).astype("int32"),2) #add color channel : (height,width,1)
          #new to old
          if INVERSE:
              row = i//3
              column = i%3
              concat[self.height*row:self.height*(row+1),self.width*column:self.width*(column+1),:] = img
          #old to new
          else:
              row =  (self.x_patch-1) - i//3
              column = (self.y_patch-1) - i%3
              concat[self.height*row:self.height*(row+1),self.width*column:self.width*(column+1),:] = img
      file_path = os.path.join(save_dir,f"{index:04d}.jpg")
      cv2.imwrite(file_path,concat)
      return file_path
    
    def make_dataset0302(self,video_idx,filename):
      #init
      count_spatter = 0
      start_frame = self.frame_data[video_idx-1][0]
      end_frame = self.frame_data[video_idx-1][1]
      spatter = self.frame_spatter[video_idx-1]
      #get num of file in a directory
      dir = r"20230302_{}".format(video_idx)
      #target directory
      src_dir = os.path.join(self.src_img_dir,dir) 
      #save directory for concatenated imgs
      save_dir = os.path.join(src_dir,self.type_dataset)
      if not os.path.exists(save_dir):
        os.makedirs(save_dir)
      #calculate num of imgs in directory  
      num_photo = sum(os.path.isfile(os.path.join(src_dir,name)) for name in os.listdir(src_dir))
      print("20230302_{} progressing: {}, start frame : {}".format(video_idx,num_photo,start_frame))
      ############
      #iteration
      if num_photo >= 1:
        for i in range(10,num_photo):
          ret = False
          if i % 50 == 49:
             print(f"{round(i/num_photo*100,1)} % progressed")
          index = start_frame + i
          #if i frame is spatter frame
          if index in spatter:
            ret=True
            #if there is no spatter frame in 9 inputs
            for j in range(1,9):
              if (index-j) in spatter:
                ret = False
           
          if ret:
            print(f"Frame {index} is spatter")
            count_spatter += 1
            #concatenate 9 imgs
            file_path = self.concatImg(index=index,
                                      root_path=src_dir,
                                      save_dir=save_dir)
            with open(filename,"a",newline='') as f:
              writer = csv.writer(f)
              writer.writerow([1,file_path]) #url starts with 0, so i-1 is adopted
        print(f"{dir} :: Number of spatter frame is {count_spatter}")
        print(f"spatter count by Kageyama-san is {len(spatter)}")
      else:
        print(f"no imgs in 20230302_{video_idx}")

## Prepare csv file

In [110]:
filename = r"C:\Users\Furukawa\Documents\TechAssistant\dataset\csv\20230302\MAE\train.csv"
filename_test= r"C:\Users\Furukawa\Documents\TechAssistant\dataset\csv\20230302\MAE\test.csv"
#///////////////////

with open(filename,"w") as f:
  writer = csv.writer(f)
  writer.writerow(["Spatter","Path"])#,"Path(t-9)","Path(t-10)","Path(t-11)","Path(t-12)","Path(t-13)","Path(t-14)","Path(t-15)","Path(t-16)","Path(t-17)"])

with open(filename_test,"w") as f:
  writer = csv.writer(f)
  writer.writerow(["Spatter","Path"])#,"Path(t-9)","Path(t-10)","Path(t-11)","Path(t-12)","Path(t-13)","Path(t-14)","Path(t-15)","Path(t-16)","Path(t-17)"])


## Make instance of Dataset Class

In [111]:
dataset = Dataset(type_dataset="concat_MAE")

## make dataset

In [112]:
for i in range(1,16):
    if i == 8:
        dataset.make_dataset0302(video_idx=i,filename=filename_test)
    else:
        dataset.make_dataset0302(video_idx=i,filename=filename)

20230302_1 progressing: 375, start frame : 1053
Frame 1067 is spatter
Frame 1082 is spatter
13.1 % progressed
26.4 % progressed
39.7 % progressed
Frame 1231 is spatter


53.1 % progressed
Frame 1262 is spatter
66.4 % progressed
Frame 1346 is spatter
79.7 % progressed
Frame 1376 is spatter
93.1 % progressed
Frame 1403 is spatter
20230302_1 :: Number of spatter frame is 7
spatter count by Kageyama-san is 7
20230302_2 progressing: 750, start frame : 2130
Frame 2148 is spatter
Frame 2166 is spatter
6.5 % progressed
Frame 2184 is spatter
Frame 2193 is spatter
Frame 2203 is spatter
Frame 2216 is spatter
13.2 % progressed
Frame 2240 is spatter
Frame 2272 is spatter
19.9 % progressed
Frame 2286 is spatter
26.5 % progressed
Frame 2331 is spatter
Frame 2357 is spatter
Frame 2370 is spatter
33.2 % progressed
Frame 2391 is spatter
39.9 % progressed
46.5 % progressed
Frame 2491 is spatter
Frame 2515 is spatter
53.2 % progressed
59.9 % progressed
Frame 2580 is spatter
Frame 2611 is spatter
66.5 % progressed
Frame 2643 is spatter
Frame 2668 is spatter
73.2 % progressed
Frame 2695 is spatter
Frame 2715 is spatter
79.9 % progressed
Frame 2735 is spatter
Frame 2752 is s

In [127]:
df_train = pd.read_csv(r"C:\Users\Furukawa\Documents\TechAssistant\dataset\csv\20230302\MAE\train.csv")
df_test = pd.read_csv(r"C:\Users\Furukawa\Documents\TechAssistant\dataset\csv\20230302\MAE\test.csv")
train = df_train.values
test = df_test.values
print(f"train data : {train.shape[0]}")
print(f"test data : {test.shape[0]}")

train data : 263
test data : 26


# Make Dataset for ViT

In [113]:
import random

class Dataset():
    def __init__(self,type_dataset,
                 height=240,width=240,x_patch=3,y_patch=3,
                 frame_data = frame_data,frame_spatter=frame_spatter,
                 src = r"C:\Users\Furukawa\Documents\TechAssistant\dataset\imgs\20230302"):
      self.type_dataset = type_dataset # type of dataset :"concat_MAE", "concat_ViT", "resnet"
      self.src_img_dir = src # source img directory
      self.height = height #height of original img
      self.width = width # width of original img
      self.x_patch = x_patch # num of patch in x axis
      self.y_patch = y_patch # num of patch in y axis
      self.frame_data = frame_data # start frame and end frame
      self.frame_spatter = frame_spatter # spatter frame

      self.H = height*y_patch #height of concatenated img
      self.W = width*x_patch # width of concatenated img 


    def concatImg(self,index,root_path,save_dir,INVERSE = False):
      """concatenate images
      Args:
        index: index for spatter frame, so index is the latest index in 9 frames, ie. index, index-1,index-2,..., index-8
        src_img_dir : source directory of cropped imgs
        save_dir : save dir for concatenate images
        x_patch, y_patch : number of patches in x and y axis
        INVERSE : patch order. If INVERSE is True, new to old is applied
      Return:
        file_path : path for concat img
        write concatenated images into designated directory
      """
      concat = np.zeros((self.H,self.W,1))
      for i in range(9):
          file = "{:04d}.jpg".format(index-i) #from new to old
          path = os.path.join(root_path,file)
          img = cv2.imread(path)
          img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #gray scale
          img = np.expand_dims(np.array(img).astype("int32"),2) #add color channel : (height,width,1)
          #new to old
          if INVERSE:
              row = i//3
              column = i%3
              concat[self.height*row:self.height*(row+1),self.width*column:self.width*(column+1),:] = img
          #old to new
          else:
              row =  (self.x_patch-1) - i//3
              column = (self.y_patch-1) - i%3
              concat[self.height*row:self.height*(row+1),self.width*column:self.width*(column+1),:] = img
      file_path = os.path.join(save_dir,f"{index:04d}.jpg")
      cv2.imwrite(file_path,concat)
      return file_path
    
    def make_dataset0302(self,video_idx,filename):
      #init
      count_spatter = 0
      count_nonspatter = 0
      start_frame = self.frame_data[video_idx-1][0]
      end_frame = self.frame_data[video_idx-1][1]
      spatter = self.frame_spatter[video_idx-1]
      #get num of file in a directory
      dir = r"20230302_{}".format(video_idx)
      #target directory
      src_dir = os.path.join(self.src_img_dir,dir) 
      #save directory for concatenated imgs
      save_dir = os.path.join(src_dir,self.type_dataset)
      if not os.path.exists(save_dir):
        os.makedirs(save_dir)
      #calculate num of imgs in directory  
      num_photo = sum(os.path.isfile(os.path.join(src_dir,name)) for name in os.listdir(src_dir))
      print("20230302_{} progressing: {}, start frame : {}".format(video_idx,num_photo,start_frame))
      ############
      #iteration
      if num_photo >= 1:
        for i in range(10,num_photo):
          ret = False
          if i % 50 == 49:
             print(f"{round(i/num_photo*100,1)} % progressed")
          index = start_frame + i
          #if there is a spatter frame in 2~10 steps
          for k in range(2,10):
            if (index+k) in spatter:
              ret=True
          #if there is no spatter frame in 9 inputs
          for j in range(9):
            if (index-j) in spatter:
              ret = False
           
          if ret:
            print(f"Frame {index} is spatter")
            count_spatter += 1
            #concatenate 9 imgs
            file_path = self.concatImg(index=index,
                                      root_path=src_dir,
                                      save_dir=save_dir)
            with open(filename,"a",newline='') as f:
              writer = csv.writer(f)
              writer.writerow([1,file_path]) #url starts with 0, so i-1 is adopted
          #non-spatter frame
          else:
            #no spatter in 2~10 frames
            for k in range(2,10):
              if (index+k) in spatter:
                continue #go to next iteration
            #no spatter in 9 input frames
            for j in range(1,9):
              if (index-j) in spatter:
                continue #go to next iteration
            #random function for variety of dataset
            if random.random() >= 0.73:
              #concatenate images
              file_path = self.concatImg(index=index,
                                      root_path=src_dir,
                                      save_dir=save_dir)
              with open(filename,"a",newline='') as f:
                writer = csv.writer(f)
                writer.writerow([0,file_path])
                count_nonspatter += 1
        print(f"{dir} :: Number of spatter frame is {count_spatter}, Non-spatter frame : {count_nonspatter}")
        print(f"spatter count by Kageyama-san is {len(spatter)}")
      else:
        print(f"no imgs in 20230302_{video_idx}")

In [114]:
root_dir = r"C:\Users\Furukawa\Documents\TechAssistant\dataset\csv\20230302\ViT"
if not os.path.exists(root_dir):
  os.makedirs(root_dir)
filename = r"C:\Users\Furukawa\Documents\TechAssistant\dataset\csv\20230302\ViT\train.csv"
filename_test= r"C:\Users\Furukawa\Documents\TechAssistant\dataset\csv\20230302\ViT\test.csv"
#///////////////////

with open(filename,"w") as f:
  writer = csv.writer(f)
  writer.writerow(["Spatter","Path"])#,"Path(t-9)","Path(t-10)","Path(t-11)","Path(t-12)","Path(t-13)","Path(t-14)","Path(t-15)","Path(t-16)","Path(t-17)"])

with open(filename_test,"w") as f:
  writer = csv.writer(f)
  writer.writerow(["Spatter","Path"])#,"Path(t-9)","Path(t-10)","Path(t-11)","Path(t-12)","Path(t-13)","Path(t-14)","Path(t-15)","Path(t-16)","Path(t-17)"])


In [115]:
dataset = Dataset(type_dataset="concat_ViT")

In [116]:
for i in range(1,16):
    if i == 8:
        dataset.make_dataset0302(video_idx=i,filename=filename_test)
    else:
        dataset.make_dataset0302(video_idx=i,filename=filename)

20230302_1 progressing: 375, start frame : 1053
Frame 1063 is spatter


Frame 1064 is spatter
Frame 1065 is spatter
Frame 1076 is spatter
Frame 1077 is spatter
Frame 1078 is spatter
Frame 1079 is spatter
Frame 1080 is spatter
13.1 % progressed
26.4 % progressed
39.7 % progressed
Frame 1222 is spatter
Frame 1223 is spatter
Frame 1224 is spatter
Frame 1225 is spatter
Frame 1226 is spatter
Frame 1227 is spatter
Frame 1228 is spatter
Frame 1229 is spatter
53.1 % progressed
Frame 1253 is spatter
Frame 1254 is spatter
Frame 1255 is spatter
Frame 1256 is spatter
Frame 1257 is spatter
Frame 1258 is spatter
Frame 1259 is spatter
Frame 1260 is spatter
66.4 % progressed
Frame 1337 is spatter
Frame 1338 is spatter
Frame 1339 is spatter
Frame 1340 is spatter
Frame 1341 is spatter
Frame 1342 is spatter
Frame 1343 is spatter
Frame 1344 is spatter
79.7 % progressed
Frame 1367 is spatter
Frame 1368 is spatter
Frame 1369 is spatter
Frame 1370 is spatter
Frame 1371 is spatter
Frame 1372 is spatter
Frame 1373 is spatter
Frame 1374 is spatter
Frame 1394 is spatter
Frame 1395 i

## Check dataset

In [117]:
df_train = pd.read_csv(filename)
train = df_train.values
label= train[:,0]
num_data = train.shape[0]
count_spatter = 0
for i in range(num_data):
    if label[i]== 1:
        count_spatter += 1
print("Train :: spatter :{},non_spatter:{}".format(count_spatter,(num_data-count_spatter)))

df_test = pd.read_csv(filename_test)
test = df_test.values
label= test[:,0]
num_data = test.shape[0]
count_spatter = 0
for i in range(num_data):
    if label[i]== 1:
        count_spatter += 1
print("spatter :{},non_spatter:{}".format(count_spatter,(num_data-count_spatter)))

Train :: spatter :1782,non_spatter:2167
spatter :160,non_spatter:158


# Make Dataset for ResNet

In [121]:
import random

#dataset is old to new
class Dataset():
    def __init__(self,
                 height=240,width=240,x_patch=3,y_patch=3,
                 frame_data = frame_data,frame_spatter=frame_spatter,
                 src = r"C:\Users\Furukawa\Documents\TechAssistant\dataset\imgs\20230302"):
      self.src_img_dir = src # source img directory
      self.height = height #height of original img
      self.width = width # width of original img
      self.x_patch = x_patch # num of patch in x axis
      self.y_patch = y_patch # num of patch in y axis
      self.frame_data = frame_data # start frame and end frame
      self.frame_spatter = frame_spatter # spatter frame

      self.H = height*y_patch #height of concatenated img
      self.W = width*x_patch # width of concatenated img 

    def make_dataset0302(self,video_idx,filename):
      #init
      count_spatter = 0
      count_nonspatter = 0
      start_frame = self.frame_data[video_idx-1][0]
      end_frame = self.frame_data[video_idx-1][1]
      spatter = self.frame_spatter[video_idx-1]
      #get num of file in a directory
      dir = r"20230302_{}".format(video_idx)
      #target directory
      src_dir = os.path.join(self.src_img_dir,dir) 
      #calculate num of imgs in directory  
      num_photo = sum(os.path.isfile(os.path.join(src_dir,name)) for name in os.listdir(src_dir))
      print("20230302_{} progressing: {}, start frame : {}".format(video_idx,num_photo,start_frame))
      ############
      #iteration
      if num_photo >= 1:
        for i in range(10,num_photo):
          ret = False
          if i % 50 == 49:
             print(f"{round(i/num_photo*100,1)} % progressed")
          index = start_frame + i
          #if there is a spatter frame in 2~10 steps
          for k in range(2,10):
            if (index+k) in spatter:
              ret=True
          #if there is no spatter frame in 9 inputs
          for j in range(9):
            if (index-j) in spatter:
              ret = False
          #file path setting
          file0 = os.path.join(src_dir,f"{index:04d}.jpg")
          file1 = os.path.join(src_dir,f"{(index-1):04d}.jpg")
          file2 = os.path.join(src_dir,f"{(index-2):04d}.jpg")
          file3 = os.path.join(src_dir,f"{(index-3):04d}.jpg")
          file4 = os.path.join(src_dir,f"{(index-4):04d}.jpg")
          file5 = os.path.join(src_dir,f"{(index-5):04d}.jpg")
          file6 = os.path.join(src_dir,f"{(index-6):04d}.jpg")
          file7 = os.path.join(src_dir,f"{(index-7):04d}.jpg")
          file8 = os.path.join(src_dir,f"{(index-8):04d}.jpg")
          file9 = os.path.join(src_dir,f"{(index-9):04d}.jpg")
          if ret:
            print(f"Frame {index} is spatter")
            count_spatter += 1
            with open(filename,"a",newline='') as f:
              writer = csv.writer(f)
              writer.writerow([1,file9,file8,file7,file6,file5,file4,file3,file2,file1,file0]) #url starts with 0, so i-1 is adopted
          #non-spatter frame
          else:
            #no spatter in 2~10 frames
            for k in range(2,10):
              if (index+k) in spatter:
                continue #go to next iteration
            #no spatter in 9 input frames
            for j in range(1,9):
              if (index-j) in spatter:
                continue #go to next iteration
            #random function for variety of dataset
            if random.random() >= 0.73:
              with open(filename,"a",newline='') as f:
                writer = csv.writer(f)
                writer.writerow([0,file9,file8,file7,file6,file5,file4,file3,file2,file1,file0])
                count_nonspatter += 1
        print(f"{dir} :: Number of spatter frame is {count_spatter}, Non-spatter frame : {count_nonspatter}")
        print(f"spatter count by Kageyama-san is {len(spatter)}")
      else:
        print(f"no imgs in 20230302_{video_idx}")

In [122]:
root_dir = r"C:\Users\Furukawa\Documents\TechAssistant\dataset\csv\20230302\resnet"
if not os.path.exists(root_dir):
  os.makedirs(root_dir)
filename = r"C:\Users\Furukawa\Documents\TechAssistant\dataset\csv\20230302\resnet\train.csv"
filename_test= r"C:\Users\Furukawa\Documents\TechAssistant\dataset\csv\20230302\resnet\test.csv"
#///////////////////

with open(filename,"w") as f:
  writer = csv.writer(f)
  writer.writerow(["Spatter","Path(t-9)","Path(t-8)","Path(t-7)","Path(t-6)","Path(t-5)","Path(t-4)","Path(t-3)","Path(t-2)","Path(t-1)","Path(t)"])

with open(filename_test,"w") as f:
  writer = csv.writer(f)
  writer.writerow(["Spatter","Path(t-9)","Path(t-8)","Path(t-7)","Path(t-6)","Path(t-5)","Path(t-4)","Path(t-3)","Path(t-2)","Path(t-1)","Path(t)"])#,"Path(t-9)","Path(t-10)","Path(t-11)","Path(t-12)","Path(t-13)","Path(t-14)","Path(t-15)","Path(t-16)","Path(t-17)"])

In [123]:
dataset = Dataset()

In [124]:
for i in range(1,16):
    if i == 8:
        dataset.make_dataset0302(video_idx=i,filename=filename_test)
    else:
        dataset.make_dataset0302(video_idx=i,filename=filename)

20230302_1 progressing: 375, start frame : 1053
Frame 1063 is spatter
Frame 1064 is spatter
Frame 1065 is spatter
Frame 1076 is spatter
Frame 1077 is spatter
Frame 1078 is spatter
Frame 1079 is spatter
Frame 1080 is spatter
13.1 % progressed
26.4 % progressed
39.7 % progressed
Frame 1222 is spatter
Frame 1223 is spatter
Frame 1224 is spatter
Frame 1225 is spatter
Frame 1226 is spatter
Frame 1227 is spatter
Frame 1228 is spatter
Frame 1229 is spatter
53.1 % progressed
Frame 1253 is spatter
Frame 1254 is spatter
Frame 1255 is spatter
Frame 1256 is spatter
Frame 1257 is spatter
Frame 1258 is spatter
Frame 1259 is spatter
Frame 1260 is spatter
66.4 % progressed
Frame 1337 is spatter
Frame 1338 is spatter
Frame 1339 is spatter
Frame 1340 is spatter
Frame 1341 is spatter
Frame 1342 is spatter
Frame 1343 is spatter
Frame 1344 is spatter
79.7 % progressed
Frame 1367 is spatter
Frame 1368 is spatter
Frame 1369 is spatter
Frame 1370 is spatter
Frame 1371 is spatter
Frame 1372 is spatter
Frame 13

## Check Dataset

In [125]:
df_train = pd.read_csv(filename)
train = df_train.values
label= train[:,0]
num_data = train.shape[0]
count_spatter = 0
for i in range(num_data):
    if label[i]== 1:
        count_spatter += 1
print("Train :: spatter :{},non_spatter:{}".format(count_spatter,(num_data-count_spatter)))

df_test = pd.read_csv(filename_test)
test = df_test.values
label= test[:,0]
num_data = test.shape[0]
count_spatter = 0
for i in range(num_data):
    if label[i]== 1:
        count_spatter += 1
print("spatter :{},non_spatter:{}".format(count_spatter,(num_data-count_spatter)))

Train :: spatter :1782,non_spatter:2145
spatter :160,non_spatter:150
